# Anlise das informações e limpeza da base de dados Viagens_2025

In [14]:
import pandas as pd
linhas_descartadas = []

In [15]:
def capturar_linha_ruim(linha):
    linhas_descartadas.append(linha)
    return None

In [16]:
df = pd.read_csv(
    "../Data Layer/raw/dados_brutos.csv",
    sep=";",
    encoding="latin1",
    na_values=["Sem informação", -11, -1],
    engine="python",
    on_bad_lines=capturar_linha_ruim
)

mask = df['Justificativa Urgência Viagem'].isna()

df.loc[mask, 'Justificativa Urgência Viagem'] = 'Sem informação'

total_linhas_lidas = len(df)

linhas_valores_invalidos = df[df.isna().any(axis=1)]
total_linhas_valores_invalidos = len(linhas_valores_invalidos)

In [17]:
colunas_financeiras = ['Valor diárias', 'Valor passagens', 'Valor devolução', 'Valor outros gastos']
for col in colunas_financeiras:
    if df[col].dtype == 'object':
        df[col] = df[col].str.replace('R$', '', regex=False)
        df[col] = df[col].str.replace('.', '', regex=False)
        df[col] = df[col].str.replace(',', '.', regex=False)
        df[col] = df[col].str.strip()

df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
df[colunas_financeiras] = df[colunas_financeiras].astype(float)
df['Total gasto'] = (df['Valor diárias'] + 
                    df['Valor passagens'] + 
                    df['Valor outros gastos'] - 
                    df['Valor devolução']).round(2)

In [18]:
df = df.dropna(axis=1, how="all")
df = df.dropna(axis=0, how="any")

In [19]:
total_linhas_descartadas = len(linhas_descartadas)
total_linhas_validas = len(df)

print("RELATÓRIO DE LIMPEZA DO CSV")
print(f"Total de linhas lidas do arquivo: {total_linhas_lidas}")
print(f"Linhas descartadas por erro estrutural: {len(linhas_descartadas)}")
print(f"Linhas descartadas por conter valores inválidos para a pesquisa: {total_linhas_valores_invalidos}")
print(f"Linhas válidas finais: {len(df)}")

RELATÓRIO DE LIMPEZA DO CSV
Total de linhas lidas do arquivo: 751057
Linhas descartadas por erro estrutural: 0
Linhas descartadas por conter valores inválidos para a pesquisa: 620526
Linhas válidas finais: 130531


In [20]:
df['Período - Data de início'] = pd.to_datetime(df['Período - Data de início'], dayfirst=True)
df['Período - Data de fim'] = pd.to_datetime(df['Período - Data de fim'], dayfirst=True)
df['Mês - Ida'] = df['Período - Data de início'].dt.month_name()
df['Mês - Volta'] = df['Período - Data de fim'].dt.month_name()

In [21]:
df.head()

,Identificador do processo de viagem,Número da Proposta (PCDP),Situação,Viagem Urgente,Justificativa Urgência Viagem,Código do órgão superior,Nome do órgão superior,Código órgão solicitante,Nome órgão solicitante,CPF viajante,...,Período - Data de fim,Destinos,Motivo,Valor diárias,Valor passagens,Valor devolução,Valor outros gastos,Total gasto,Mês - Ida,Mês - Volta
2,20286869,000002/25,Realizada,NÃO,Sem informação,26000.0,Ministério da Educação,26241.0,Universidade Federal do Paraná,***.235.999-**,...,2025-02-01,Palmas/TO,PARTICIPAÇÃO DO XXV ENCONTRO BRASILEIRO DE ICT...,0.0,2544.74,0.0,0.00,2544.74,January,February
71,20372222,000002/25,Realizada,NÃO,Sem informação,26000.0,Ministério da Educação,26250.0,Universidade Federal de Roraima,***.690.937-**,...,2025-02-01,Manaus/AM,Participar da reunião anual do PPG- Bionorte q...,0.0,961.43,0.0,0.00,961.43,January,February
79,20378060,000001/25,Realizada,NÃO,Sem informação,26000.0,Ministério da Educação,26350.0,Fundação Universidade Federal da Grande Dourados,***.715.828-**,...,2025-04-06,Belo Horizonte/MG,Participar do XXIV Congresso Brasileiro de Ger...,0.0,866.64,0.0,0.00,866.64,April,April
138,20452941,000001/25-1C,Realizada,NÃO,Sem informação,26000.0,Ministério da Educação,26247.0,Universidade Federal de Santa Maria,***.636.670-**,...,2025-02-04,"Valladolid/Espanha, Valladolid/Espanha",Participar do Seminário Internacional de Prax...,0.0,6825.25,0.0,0.00,6825.25,January,February
139,20459695,000003/25,Realizada,NÃO,Sem informação,26000.0,Ministério da Educação,26271.0,Fundação Universidade de Brasília,***.926.526-**,...,2025-02-02,Salamanca/Espanha,O docente participará do Congresso Internacion...,0.0,9134.82,0.0,200.55,9335.37,January,February


In [22]:
df.to_csv(
    "../Data Layer/silver/Viagens_2025_LIMPAS.csv",
    index=False,
    sep=";",
    encoding="latin1",
    decimal=",",
    date_format="%d/%m/%Y",
)

In [115]:
mapa_colunas = {
    'Identificador do processo de viagem': 'ide_pro_via',
    'Número da Proposta (PCDP)': 'nmr_ppt_pcd ',
    'Situação': 'situacao',
    'Viagem Urgente': 'via_urg',
    'Justificativa Urgência Viagem': 'jus_urg_via',
    'Código do órgão superior': 'cod_org_sup',
    'Nome do órgão superior': 'nom_org_sup',
    'Código órgão solicitante': 'cod_org_sol',
    'Nome órgão solicitante': 'cod_org_sol',
    'CPF viajante': 'cpf_vjt',
    'Nome': 'nome',
    'Cargo': 'cargo',
    'Descrição Função': 'funcao',
    'Período - Data de início': 'dat_ini',
    'Período - Data de fim': 'dat_fim',
    'Destinos': 'destinos',
    'Motivo': 'motivo',
    'Valor diárias': 'val_dia',
    'Valor passagens': 'val_pas',
    'Valor devolução': 'val_dev',
    'Valor outros gastos': 'val_out_gas',
    'Total gasto': 'tot_gas',
    'Mês - Ida': 'mes_ida',
    'Mês - Volta': 'mes_vol',
}


In [116]:
df = df.rename(columns=mapa_colunas)